## Preprocessing: Data Cleaning & EDA

In [113]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import string
import re
import nltk

from tqdm import trange
from nltk import tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder

In [114]:
df = pd.read_csv('data/renting.csv')

### 1. Cleaning the data

Removing duplicated rows:

In [115]:
num_rows = df.shape[0]
df.drop_duplicates(inplace=True)
print(f'Dropped {num_rows - df.shape[0]} duplicate rows')

Dropped 2303 duplicate rows


Removing non-useful columns:

In [116]:
df.drop('_id', axis=1, inplace=True)
df.drop('Useful square meters', axis=1, inplace=True)
df.drop('Orientation', axis=1, inplace=True)
df.drop('Real state', axis=1, inplace=True)

In [117]:
print("Number of rows of the column 'Plot square meters' with NaN values: ", df['Plot square meters'].isna().sum())

Number of rows of the column 'Plot square meters' with NaN values:  8086


Plot square meters has a great number of NaN as it's only valuable for houses, not for apartments. Therefore, we'll fill with 0s the NaN values. This will also be useful to distinct between houses and apartments.

In [118]:
df['Plot square meters'].fillna(0, inplace=True)

Check what are the columns with the most NaN values:

In [119]:
df.isna().sum()

Title                     0
Description              96
Price                     0
Deposit                3672
Type                      1
Built square meters       0
Plot square meters        0
Rooms                     0
Bathrooms                 0
Storage room              0
Wardrobe                  0
Furnished                 0
Equipped kitchen          0
Renovation                0
Reduced mobility          0
Heating                   0
Garage                    0
Floor                   930
Elevator                  0
Air conditioning          0
Swimming pool             0
Garden                    0
Green areas               0
Terrace                   0
Consumption            4201
Emissions              4786
Location                  0
dtype: int64

Drop the row that has a NaN value in the column 'Type':

In [120]:
df.dropna(subset=['Type'], inplace=True)

Show a bar plot where each X axis value is a value from the 'Type' column and the Y axis is the number of samples that have NaN in the 'Floor' column:

In [121]:
df['Floor'].isna().groupby(df['Type']).sum().plot(kind='bar')

<Axes: xlabel='Type'>

Therefore, we will remove the rows that have NaN in the 'Floor' column and the value 'piso' in the 'Type' column:

In [122]:
df.drop(df[(df['Floor'].isna()) & (df['Type'] == 'piso')].index, inplace=True)

Now, we're sure that there are no apartments with NaN values in the 'Floor' column. So, for the rest of the NaN values in the 'Floor' column, we will replace them with the value 'no aplica':

In [123]:
df['Floor'].fillna('no aplica', inplace=True)

Print the number of rows where the columns 'Consumption' and 'Emissions' have NaN values at the same time:

In [124]:
# Print the number of rows where the columns 'Consumption' and 'Emissions' have NaN values at the same time
print("Number of rows with NaN values in the columns 'Consumption' and 'Emissions': ", df[(df['Consumption'].isna()) & (df['Emissions'].isna())].shape[0])

Number of rows with NaN values in the columns 'Consumption' and 'Emissions':  4152


Therefore, we will remove the columns 'Consumption' and 'Emissions':

In [125]:
df.drop('Consumption', axis=1, inplace=True)
df.drop('Emissions', axis=1, inplace=True)

Finally, we're going to join the columns 'Title' and 'Description' into one column called 'Description' to avoid data sparsity:

In [126]:
df['Description'] = df.apply(lambda x: x['Title'] if pd.isna(x['Description']) else x['Title'] + '. ' + x['Description'], axis=1)
df.drop('Title', axis=1, inplace=True)

In [127]:
df.isna().sum()

Description               0
Price                     0
Deposit                3464
Type                      0
Built square meters       0
Plot square meters        0
Rooms                     0
Bathrooms                 0
Storage room              0
Wardrobe                  0
Furnished                 0
Equipped kitchen          0
Renovation                0
Reduced mobility          0
Heating                   0
Garage                    0
Floor                     0
Elevator                  0
Air conditioning          0
Swimming pool             0
Garden                    0
Green areas               0
Terrace                   0
Location                  0
dtype: int64

Now remove the samples that are in English:
Using common Eng Expressions

Based on the analysis of English comments throughtout descriptions, there aren't any many (if any) description fully in English, only small fragments at the start to attract user attention.
Therefore we do not believe it is needed to delete the comments.

In [128]:
english_keywords = ['the', 'and', 'is', 'are', 'was', 'were', 'be', 'been', 'has', 'have', 'will', 'shall']

def contains_english(text):
    text_words = text.lower().split()
    return any(word in text_words for word in english_keywords)

df_english = df[df['Description'].apply(contains_english)]
df_english['Description'].head()
df_english.to_csv('data/renting_english.csv', index=False)


#### Deposit to int

In [129]:
def extract_first_number(text):
    if pd.isna(text) or not isinstance(text, str):
        return 0
    matches = re.findall(r'\d+', text)
    return int(matches[0]) if matches else 0

df['Deposit'] = df['Deposit'].apply(extract_first_number)


### 2. Exploratory Data Analysis

#### Histogram of the price of the houses

In [130]:
fig = px.histogram(df, x='Price', title='Histogram of the price of the houses')
fig.show()

#### Distribution of prices by Type of property

In [131]:
fig = px.box(df, y='Type', x='Price', title='Distribution of prices by Type of property')
fig.show()

#### Heat Map of Corelation Between Attribtues
Eventhought theres no direct correlation between Plot Square Meters and Price, there could be Correlation if "Plot square meters is taken into account together with "Built square meters".

In [132]:
import plotly.graph_objects as go

df_numeric = df.select_dtypes(include=[np.number])

corr = df_numeric.corr()

fig = go.Figure(data=go.Heatmap(
    z=corr,
    x=corr.columns,
    y=corr.columns,
    colorscale='Blues'
))
fig.show()

#### Scatterplot of Deposit with respect to Price
We can see that deposit does not correlate too much to the price

In [133]:
fig = px.scatter(df, y='Deposit', x='Price', title='Scatter Plot de Depósito vs Precio')

fig.update_layout(
    xaxis_title="Depósito",
    yaxis_title="Precio",
    xaxis=dict(
        title_font=dict(size=14),
        tickfont=dict(size=12),
    ),
    yaxis=dict(
        title_font=dict(size=14),
        tickfont=dict(size=12),
    ),
    title=dict(
        x=0.5,
        y=0.95,
        font=dict(
            family="Courier New, monospace",
            size=20,
            color="#7f7f7f"
        )
    )
)

fig.show()


#### Removal of Deposit

In [134]:
df.drop('Deposit', axis=1, inplace=True)


#### Instances by Type of Home
We could delete the everything else apart from "piso" as they do not have a population big enough.

In [135]:
import plotly.express as px

type_counts = df['Type'].value_counts()

fig = px.bar(type_counts, title='Quantity of Instances per Type of Home')
fig.update_xaxes(type='category')
fig.update_layout(
    xaxis_title="Tipo de Vivienda",
    yaxis_title="Cantidad de Instancias",
    xaxis=dict(
        title_font=dict(size=14),
        tickfont=dict(size=12),
    ),
    yaxis=dict(
        title_font=dict(size=14),
        tickfont=dict(size=12),
    ),
    title=dict(
        x=0.5,
        y=0.95,
        font=dict(
            family="Courier New, monospace",
            size=20,
            color="#7f7f7f"
        )
    )
)
fig.show()


#### Boxplot of Price by House Type
We can see that they all tend to have a similar box range. 
The range of prices boxes varies but we have to remember the difference of number of instances of the Type 

In [136]:
fig = px.box(df, y='Type', x='Price', title='Distribución de Precios por Tipo de Casa')

fig.update_layout(
    xaxis_title="Tipo de Casa",
    yaxis_title="Precio",
    xaxis=dict(
        title_font=dict(size=14),
        tickfont=dict(size=12),
    ),
    yaxis=dict(
        title_font=dict(size=14),
        tickfont=dict(size=12),
    ),
    title=dict(
        x=0.5,
        y=0.95,
        font=dict(
            family="Courier New, monospace",
            size=20,
            color="#7f7f7f"
        )
    )
)



#### Removal the values of 'Type' that are not necessary:

Based on observations, the prices do vary, but due to the size difference between 'piso' and the rest, they could be removed as we see they do not pose a significant difference.

In [137]:
init_shape = df.shape[0]
df_filtered = df[df['Type'] == 'piso']
fini_shape = df_filtered.shape[0]

# Remove comment if we want to filter out the other types
#df = df[df['Type'] == 'piso']

print(f'Number of rows with Type = "piso": {fini_shape}, other types: {init_shape - fini_shape}')

Number of rows with Type = "piso": 6239, other types: 2096


#### Scatterplot Deposit vs Price
Deposit already deleted

In [138]:
"""
fig = px.scatter_mapbox(df, lat='Latitude', lon='Longitude', color='Price',
                        size='Price', size_max=15, zoom=10,
                        mapbox_style="carto-positron", title='Mapa de Calor de Precios en Madrid')

fig.show()
"""

'\nfig = px.scatter_mapbox(df, lat=\'Latitude\', lon=\'Longitude\', color=\'Price\',\n                        size=\'Price\', size_max=15, zoom=10,\n                        mapbox_style="carto-positron", title=\'Mapa de Calor de Precios en Madrid\')\n\nfig.show()\n'

### 3. Exporting the data

In [139]:
df.to_csv('data/preprocessed_data.csv', index=False)